In [3]:
import pandas as pd
import csv
import os

# 1. Abrindo o arquivo routes.txt

O arquivo routes.txt contém as informações sobre o letreiro da rota, cor e id.

In [4]:
routes = pd.read_csv('gtfs/routes.txt', usecols=['route_id', 'route_long_name','route_color'])
routes

,route_id,route_long_name,route_color
0,1012-10,Term. Jd. Britania - Jd. Monte Belo,509E2F
1,1012-21,Jd. Rosinha - Term. Jd. Britânia,509E2F
2,1015-10,Term. Jd. Britânia - Chác. Maria Trindade,509E2F
3,1016-10,Cem. Do Horto - Shop. Center Norte,002F6C
4,1017-10,Perus - Conexão Vl. Iório,509E2F
...,...,...,...
1361,N839-11,Metrô Butantã - Metrô Vl. Mariana,FF671F
1362,N840-11,Term. Vl. Mariana - Sta. Cecília,FF671F
1363,N841-11,Term. Vl. Mariana - Sta. Cecília,FF671F
1364,N842-11,Term. Pinheiros - Cohab Raposo Tavares,FF671F


### Contando a quantidade de cores que aparecem

In [5]:
colors = routes['route_color'].value_counts()
colors

0082BA    210
782F40    193
FFD100    184
509E2F    163
DA291C    162
002F6C    161
006341    141
FF671F    139
00B352      1
FFFF00      1
EE372F      1
007E5E      1
0455A1      1
C0C0C0      1
01A9A7      1
133C8D      1
F68368      1
049FC3      1
97A098      1
CA016B      1
9B3894      1
Name: route_color, dtype: int64

### Selecionando índices com valores maiores que 1

In [6]:
colors = colors[colors.values > 1]
colorlist = colors.index.to_list()
colorlist


# mostrando as cores
# https://gist.github.com/wmayner/9b099a0e4a5f8e94f0c6ab2f570187a5

from IPython.display import Markdown, display

display(Markdown('<br>'.join(
    f'<span style="font-family: monospace">#{color} <span style="color: #{color}">████████</span></span>'
    for color in colorlist
)))

<span style="font-family: monospace">#0082BA <span style="color: #0082BA">████████</span></span><br><span style="font-family: monospace">#782F40 <span style="color: #782F40">████████</span></span><br><span style="font-family: monospace">#FFD100 <span style="color: #FFD100">████████</span></span><br><span style="font-family: monospace">#509E2F <span style="color: #509E2F">████████</span></span><br><span style="font-family: monospace">#DA291C <span style="color: #DA291C">████████</span></span><br><span style="font-family: monospace">#002F6C <span style="color: #002F6C">████████</span></span><br><span style="font-family: monospace">#006341 <span style="color: #006341">████████</span></span><br><span style="font-family: monospace">#FF671F <span style="color: #FF671F">████████</span></span>

Observe que existem exatamente 8 tipos de cores de linhas/regiões

### Filtrando todas as rotas com cores selecionadas

In [7]:
lines = routes.loc[routes["route_color"].isin(colorlist)]
lines

,route_id,route_long_name,route_color
0,1012-10,Term. Jd. Britania - Jd. Monte Belo,509E2F
1,1012-21,Jd. Rosinha - Term. Jd. Britânia,509E2F
2,1015-10,Term. Jd. Britânia - Chác. Maria Trindade,509E2F
3,1016-10,Cem. Do Horto - Shop. Center Norte,002F6C
4,1017-10,Perus - Conexão Vl. Iório,509E2F
...,...,...,...
1361,N839-11,Metrô Butantã - Metrô Vl. Mariana,FF671F
1362,N840-11,Term. Vl. Mariana - Sta. Cecília,FF671F
1363,N841-11,Term. Vl. Mariana - Sta. Cecília,FF671F
1364,N842-11,Term. Pinheiros - Cohab Raposo Tavares,FF671F


O dataframe de rotas está pronto e contém todas as informações relevantes. Agora será necessário relacioná-lo com as informações do arquivo trips.txt através da coluna 'route_id'.

# 2. Abrindo o arquivo trips.txt

O arquivo trips.txt contém as informações sobre a direção de uma rota (ida e volta), além do id das coordenadas do percurso.

In [8]:
trips = pd.read_csv('gtfs/trips.txt', usecols=['route_id', 'trip_headsign', 'direction_id', 'shape_id'])

trips

,route_id,trip_headsign,direction_id,shape_id
0,1012-10,Jd. Monte Belo,0,72757
1,1012-10,Term. Jd. Britania,1,72758
2,1012-21,Term. Jd. Britânia,0,79152
3,1015-10,Chác. Maria Trindade,0,78289
4,1016-10,Shop. Center Norte,0,72283
...,...,...,...,...
2235,N839-11,Metrô Vl. Mariana,0,63767
2236,N840-11,Sta. Cecília,0,79253
2237,N841-11,Sta. Cecília,0,68304
2238,N842-11,Cohab Raposo Tavares,0,78326


### Relacionando e mesclando a tabela 'lines' com 'trips'

In [9]:
lines = lines.merge(trips[['route_id', 'direction_id', 'shape_id']], left_on='route_id', right_on='route_id', 
                    suffixes=(False, False))
lines

,route_id,route_long_name,route_color,direction_id,shape_id
0,1012-10,Term. Jd. Britania - Jd. Monte Belo,509E2F,0,72757
1,1012-10,Term. Jd. Britania - Jd. Monte Belo,509E2F,1,72758
2,1012-21,Jd. Rosinha - Term. Jd. Britânia,509E2F,0,79152
3,1015-10,Term. Jd. Britânia - Chác. Maria Trindade,509E2F,0,78289
4,1016-10,Cem. Do Horto - Shop. Center Norte,002F6C,0,72283
...,...,...,...,...,...
2209,N839-11,Metrô Butantã - Metrô Vl. Mariana,FF671F,0,63767
2210,N840-11,Term. Vl. Mariana - Sta. Cecília,FF671F,0,79253
2211,N841-11,Term. Vl. Mariana - Sta. Cecília,FF671F,0,68304
2212,N842-11,Term. Pinheiros - Cohab Raposo Tavares,FF671F,0,78326


Observe que as linhas estão duplicadas na coluna 'route_long_name', isso aconteceu porque a maioria das rotas possuem dois trajetos.
Trajeto principal (ida ou 0);
trajeto secundário (volta ou 1).

# 3. Abrindo o arquivo shapes.txt

O arquivo shapes.txt contém informações e coordenadas (trajeto) de todas as rotas.

In [10]:
shapes = pd.read_csv('gtfs/shapes.txt', usecols=['shape_id', 'shape_dist_traveled'])
shapes

,shape_id,shape_dist_traveled
0,72757,0.000000
1,72757,6.504682
2,72757,65.418427
3,72757,78.136925
4,72757,82.364815
...,...,...
1120416,62557,24200.707000
1120417,62557,24289.654000
1120418,62557,24308.768000
1120419,62557,24326.680000


### Obtendo a última ocorrência do id do trajeto para obter o comprimento do trajeto

In [11]:
routlen = shapes.drop_duplicates(subset='shape_id', keep='last')
routlen

,shape_id,shape_dist_traveled
276,72757,6961.7739
555,72758,6597.3252
703,79152,3999.8555
1066,78289,15000.8890
1461,72283,11070.9740
...,...,...
1116957,63767,24604.0900
1117307,79253,17224.8360
1117668,68304,15832.8140
1119343,78326,46637.7580


In [12]:
lines = lines.merge(routlen, left_on='shape_id', right_on='shape_id', suffixes=(False, False))
lines

,route_id,route_long_name,route_color,direction_id,shape_id,shape_dist_traveled
0,1012-10,Term. Jd. Britania - Jd. Monte Belo,509E2F,0,72757,6961.7739
1,1012-10,Term. Jd. Britania - Jd. Monte Belo,509E2F,1,72758,6597.3252
2,1012-21,Jd. Rosinha - Term. Jd. Britânia,509E2F,0,79152,3999.8555
3,1015-10,Term. Jd. Britânia - Chác. Maria Trindade,509E2F,0,78289,15000.8890
4,1016-10,Cem. Do Horto - Shop. Center Norte,002F6C,0,72283,11070.9740
...,...,...,...,...,...,...
2209,N839-11,Metrô Butantã - Metrô Vl. Mariana,FF671F,0,63767,24604.0900
2210,N840-11,Term. Vl. Mariana - Sta. Cecília,FF671F,0,79253,17224.8360
2211,N841-11,Term. Vl. Mariana - Sta. Cecília,FF671F,0,68304,15832.8140
2212,N842-11,Term. Pinheiros - Cohab Raposo Tavares,FF671F,0,78326,46637.7580


Agora os arquivos GTFS não serão mais necessários, com exceção do shapes.txt que será referenciado nesta tabela.

### Separando linhas diurnas de linhas noturnas e exportando

In [13]:
dtable = lines.loc[lines['route_id'].str[0] != 'N']
ntable = lines.loc[lines['route_id'].str[0] == 'N']

if not os.path.exists('./tables'):
    os.mkdir('./tables')

dtable.to_csv('./tables/dlines.csv', index=False)
ntable.to_csv('./tables/nlines.csv', index=False)

## 4. Filtrando os destinos (ida/volta) e exportando

In [14]:
#diurna
ddest = trips.loc[trips['route_id'].isin(dtable['route_id'])]
ddest = ddest['trip_headsign'].value_counts()
ddest.to_csv('./tables/ddest.csv', header=False)
#noturna
ndest = trips.loc[trips['route_id'].isin(ntable['route_id'])]
ndest = ndest['trip_headsign'].value_counts()
ndest.to_csv('./tables/ndest.csv', header=False)